# Local RAG with Ollama and LanceDB

In [2]:
import ollama

## Chatting with no context

In [3]:
question = "What were the iPhone's best new features"
stream = ollama.chat(
  model="llama3.1", stream=True,
  messages = [
    { "role": "user", 'content': f"Question: {question}"}
  ]
)
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

The answer to this question can vary depending on the specific model of iPhone and release date, but I'll focus on some notable releases.

Here are some of the most significant new features introduced in popular iPhone models:

1. **iPhone 4 (2010)**:
	* A new design with a stainless steel frame and glass front and back.
	* A 5MP camera with LED flash.
	* HD video recording capabilities.
2. **iPhone 4S (2011)**:
	* Siri, an intelligent personal assistant that allows voice commands.
	* Improved cameras with 8MP stills and 1080p video.
	* Dual-core processor for faster performance.
3. **iPhone 5 (2012)**:
	* A larger 4-inch screen with a new Lightning connector.
	* Better cameras, including a 8MP camera with improved low-light performance.
	* New features like iCloud and Passbook.
4. **iPhone 6 and iPhone 6 Plus (2014)**:
	* Larger screens (4.7 inches for the standard model, 5.5 inches for the Plus).
	* Improved cameras with a new front-facing camera (2MP) and a rear camera with optical 

## Ingesting data

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("data/sentences.csv")

In [7]:
df.iloc[0]


url         https://www.bbc.com/news/articles/clylqly0n28o
title    Bedford Debenhams plan: 'We need a reason to c...
index                                                    0
text     Shoppers have said they hope a council's plans...
Name: 0, dtype: object

In [8]:
import lancedb
import time
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import EmbeddingFunctionRegistry

/Users/markhneedham/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
registry = EmbeddingFunctionRegistry.get_instance()
embedder = registry.get("ollama").create(name="mxbai-embed-large")

In [10]:
class Schema(LanceModel):
  text: str = embedder.SourceField()
  vector: Vector(embedder.ndims()) = embedder.VectorField()
  index: int
  title: str
  url: str

In [11]:
db = lancedb.connect("./lancedb")
table = db.create_table("sentences", schema=Schema)

In [12]:
%%time
table.add(df)

CPU times: user 1.7 s, sys: 145 ms, total: 1.84 s
Wall time: 38.5 s


In [15]:
table.to_pandas().iloc[0]


text      Shoppers have said they hope a council's plans...
vector    [-0.36013725, 0.2258691, -0.62453365, 0.441479...
index                                                     0
title     Bedford Debenhams plan: 'We need a reason to c...
url          https://www.bbc.com/news/articles/clylqly0n28o
Name: 0, dtype: object

## Querying LanceDB

In [17]:
from utils import view

In [18]:
rows = (table
  .search(question)
  .limit(3)
  .to_pydantic(Schema)
)

view(rows)

[
    Schema(
        text='"The combination of Apple Intelligence and new camera features on the iPhone 16 will help spur 
upgrades from loyal Apple customers," he said. "Particularly as Apple is positioning this latest update as being a 
future-proof purchase for customers wanting to get Apple Intelligence features as they roll out over the next few 
years."',
        vector=FixedSizeList(dim=1024),
        index=1247,
        title='Apple banks on AI to boost sales of new iPhone 16 - BBC News',
        url='https://www.bbc.com/news/articles/c8erzx2n3j7o'
    ),
    Schema(
        text='Apple said its new phones, which come with longer lasting batteries, more powerful chips and enhanced
privacy features, were its first built specifically to handle AI and its new "Apple Intelligence" tools, many of 
which were announced in June. ',
        vector=FixedSizeList(dim=1024),
        index=1235,
        title='Apple banks on AI to boost sales of new iPhone 16 - BBC News',
        url='htt

In [19]:
def extract_context(rows):
  return sorted(
    [
      {"title": r.title, "text": r.text, "index": r.index} 
      for r in rows
    ], 
    key=lambda x: x['index']
  )

In [20]:
view(extract_context(rows))

[
    {
        'title': 'Apple banks on AI to boost sales of new iPhone 16 - BBC News',
        'text': 'The button is an external clue to the changes Apple said it had made inside its latest smartphone,
aimed at harnessing the latest in artificial intelligence (AI).',
        'index': 1231
    },
    {
        'title': 'Apple banks on AI to boost sales of new iPhone 16 - BBC News',
        'text': 'Apple said its new phones, which come with longer lasting batteries, more powerful chips and 
enhanced privacy features, were its first built specifically to handle AI and its new "Apple Intelligence" tools, 
many of which were announced in June. ',
        'index': 1235
    },
    {
        'title': 'Apple banks on AI to boost sales of new iPhone 16 - BBC News',
        'text': '"The combination of Apple Intelligence and new camera features on the iPhone 16 will help spur 
upgrades from loyal Apple customers," he said. "Particularly as Apple is positioning this latest update as being a 
f

## Chatting with context

In [21]:
SYSTEM = """
You will receive paragraphs of text from a news article. 
Answer the subsequent question using that context.
"""

In [22]:
question = "What were the iPhone's best new features"
rows = (table.search(question).limit(3).to_pydantic(Schema))
context = extract_context(rows)
stream = ollama.chat(
  model="llama3.1", stream=True,
  messages = [
    { "role": "system", 'content': SYSTEM},
    { "role": "user", 'content': f"Context: {context}"},
    { "role": "user", 'content': f"Question: {question}"}
  ]
)
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

According to the text, the new features of the iPhone include:

1. Longer lasting batteries
2. More powerful chips
3. Enhanced privacy features
4. A camera with enhanced features (specifically mentioned as being part of the combination that will help spur upgrades)